In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset

In [14]:
from ray import tune

In [3]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")

In [4]:
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)
updated_df

,value
date,
2022-02-05,1092.0
2022-02-12,1164.0
2022-02-19,1493.0
2022-02-26,1603.0
2022-03-05,1791.0
...,...
2025-01-18,32561.0
2025-01-25,39927.0
2025-02-01,50272.0


In [5]:
#Let's create an AutoLSTM model that is trained up to October 5, fix these parameters and create expanding window models
oct1stweekdf = updated_df.loc[:"2024-10-05"]
oct1stweekdf['ds'] = oct1stweekdf.index
oct1stweekdf["unique_id"] = "series_1"
oct1stweekdf = oct1stweekdf.rename(columns = {"value": "y"})
oct1stweekdf

,y,ds,unique_id
date,,,
2022-02-05,1092.0,2022-02-05,series_1
2022-02-12,1164.0,2022-02-12,series_1
2022-02-19,1493.0,2022-02-19,series_1
2022-02-26,1603.0,2022-02-26,series_1
2022-03-05,1791.0,2022-03-05,series_1
...,...,...,...
2024-09-07,223.0,2024-09-07,series_1
2024-09-14,319.0,2024-09-14,series_1
2024-09-21,293.0,2024-09-21,series_1


In [13]:
dataset, *_ = TimeSeriesDataset.from_df(oct1stweekdf)

In [15]:
#Batch Size Exploration
config = dict(batch_size = tune.choice([8, 16, 32, 64]), encoder_hidden_size = 200, encoder_n_layers = 2, context_size = 10, decoder_hidden_size = 200, learning_rate = 0.001, max_steps = 1000, input_size = -1, inference_input_size = -1)
LSTMmodel = AutoLSTM(h = 4, backend = "ray", config = config)
LSTMmodel.fit(dataset = dataset)

(_train_tune pid=515532) /home/hmf6av/.local/lib/python3.11/site-packages/ray/tune/integration/pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=515532) [rank: 0] Seed set to 1
(_train_tune pid=515532) GPU available: True (cuda), used: True
(_train_tune pid=515532) TPU available: False, using: 0 TPU cores
(_train_tune pid=515532) HPU available: False, using: 0 HPUs
(_train_tune pid=515532) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(_train_tune pid=515532) 
(_train_tune pid=515532)   | Name            | Type          | Params | Mode 
(_train_tune pid=515532) ----------------------------------------------------------
(_train_tune pid=515532) 0 | loss            | MAE           | 0      | train
(_train_tune pid=515532) 1 | padder          | ConstantPad1d | 0      | train
(_train_tune pid=515532) 2 | scaler          | TemporalNorm  | 0      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 76.85it/s, v_num=2587929, train_loss_step=0.418, train_loss_epoch=0.450]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 177.06it/s]
(_train_tune pid=515532) 
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 75.07it/s, v_num=2587929, train_loss_step=0.162, train_loss_epoch=0.174, valid_loss=677.0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 77.00it/s, v_num=2587929, train_loss_step=0.113, train_loss_epoch=0.0989, valid_loss=159.0] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 73.98it/s, v_num=2587929, train_loss_step=0.106, train_loss_epoch=0.106, valid_loss=113.0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 76.10it/s, v_num=2587929, train_loss_step=0.0961, train_loss_epoch=0.090, valid_loss=112.0

(_train_tune pid=515532) `Trainer.fit` stopped: `max_steps=1000` reached.
(_train_tune pid=515629) /home/hmf6av/.local/lib/python3.11/site-packages/ray/tune/integration/pytorch_lightning.py:198: `ray.tune.integration.pytorch_lightning.TuneReportCallback` is deprecated. Use `ray.tune.integration.pytorch_lightning.TuneReportCheckpointCallback` instead.
(_train_tune pid=515629) [rank: 0] Seed set to 1
(_train_tune pid=515629) GPU available: True (cuda), used: True
(_train_tune pid=515629) TPU available: False, using: 0 TPU cores
(_train_tune pid=515629) HPU available: False, using: 0 HPUs
(_train_tune pid=515629) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(_train_tune pid=515629) 
(_train_tune pid=515629)   | Name            | Type          | Params | Mode 
(_train_tune pid=515629) ----------------------------------------------------------
(_train_tune pid=515629) 0 | loss            | MAE           | 0      | train
(_train_tune pid=515629) 1 | padder          | ConstantPad1d | 0      | tr

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 76.12it/s, v_num=2587929, train_loss_step=0.418, train_loss_epoch=0.450]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 75.90it/s, v_num=2587929, train_loss_step=0.162, train_loss_epoch=0.174, valid_loss=677.0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 76.12it/s, v_num=2587929, train_loss_step=0.113, train_loss_epoch=0.0989, valid_loss=159.0] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 399: 100%|██████████| 1/1 [00:00<00:00, 77.26it/s, v_num=2587929, train_loss_step=0.106, train_loss_epoch=0.106, valid_loss=113.0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 77.12it/s, v_num=2587929, train_loss_step=0.0961, train_loss_epoch=0.090, valid_loss=112.0] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 599: 100%|██████████| 1/1 [00:00<00:00, 76.

(_train_tune pid=515629) `Trainer.fit` stopped: `max_steps=1000` reached.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


AutoLSTM